### 1. Client herunterladen

In [2]:
!git clone https://gitlab.hs-osnabrueck.de/agrifood-tef/edge-benchmarking-client.git \
    && cd edge-benchmarking-client

Cloning into 'edge-benchmarking-client'...
Username for 'https://gitlab.hs-osnabrueck.de': ^C


### 2. Client installieren

In [1]:
!pip install ..

Processing /home/jovyan/work/edge-benchmarking-client
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 13.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.1 MB/s eta 0:00:00
  Created wheel for edge-benchmarking-client: filename=edge_benchmarking_client-0.1.0-py3-none-any.whl size=3943 sha256=49f208316db172a7b01dc088d6813552cdba5d3a40bda03f16b93eeff7d31dbd
  Stored in directory: /home/jovyan/.cache/pip/wheels/b4/4e/18/ab2135139498b772545d24413e40a1d1c14c7b98f1361db9f9
Successfully built edge-benchmarking-client
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Succes

### 3. Client erstellen

In [80]:
import os
from dotenv import load_dotenv
from edge_benchmarking_client.client import EdgeBenchmarkingClient

In [81]:
load_dotenv()

# Connection information
PROTOCOL = "https"
HOST = "api.edge-farm.agrifood-tef.edvsz.hs-osnabrueck.de"

# Basic API authentication
BASIC_AUTH_USERNAME = os.getenv("BASIC_AUTH_USERNAME")
BASIC_AUTH_PASSWORD = os.getenv("BASIC_AUTH_PASSWORD")

# Create the client
client = EdgeBenchmarkingClient(
    protocol=PROTOCOL,
    host=HOST,
    username=BASIC_AUTH_USERNAME,
    password=BASIC_AUTH_PASSWORD,
)

2024-07-04 14:19:10.538 INFO client - __init__: Created Edge Benchmarking client for user 'admin' and Edge Farm at 'https://api.edge-farm.agrifood-tef.edvsz.hs-osnabrueck.de'.
2024-07-04 14:19:10.603 INFO client - _test_connection: Edge Farm API is reachable with status code 200.


### 4. Benchmark-Daten finden und einlesen

In [82]:
EXAMPLE_ROOT_DIR = "densenet_onnx"

dataset = client.find_dataset(
    root_dir=EXAMPLE_ROOT_DIR, file_extensions={".jpg", ".jpeg"}
)
model = client.find_model(root_dir=EXAMPLE_ROOT_DIR)
model_metadata = client.find_model_metadata(root_dir=EXAMPLE_ROOT_DIR)
labels = client.find_labels(root_dir=EXAMPLE_ROOT_DIR)

2024-07-04 14:19:12.842 INFO client - find_dataset: Found dataset containing 6 samples with type(s) {'.jpg', '.jpeg'} in '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx'.
2024-07-04 14:19:12.850 INFO client - _find_file: Found file '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx/model/densenet_onnx.onnx' with extension {'.pt', '.pth', '.onnx'} in '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx'.
2024-07-04 14:19:12.855 WARNING client - _find_file: Found multiple (2) files with extension {'.pbtxt'} in '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx'. Selecting first match.
2024-07-04 14:19:12.856 INFO client - _find_file: Found file '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx/model/config.pbtxt' with extension {'.pbtxt'} in '/home/jovyan/work/edge-benchmarking-client/examples/densenet_onnx'.
2024-07-04 14:19:12.864 WARNING client - _find_file: Found multiple (2) files with extension {'.txt

### 5. Benchmark-Job starten und auf Ergebnisse warten

In [83]:
benchmark_results, inference_results = client.benchmark(
    edge_device="edge-03",
    dataset=dataset,
    model=model,
    model_name=EXAMPLE_ROOT_DIR,
    model_metadata=model_metadata,
    labels=labels,
    num_classes=1000,
)

2024-07-04 14:19:23.556 INFO client - upload_benchmark_data: 200 - {'bucket_name': 'a0713a6a-17fd-4de5-8e63-3264e9ea5b16', 'dataset': ['dataset/mug.jpg', 'dataset/car.jpg', 'dataset/car-checkpoint.jpg', 'dataset/mug-checkpoint.jpg', 'dataset/vulture.jpeg', 'dataset/vulture-checkpoint.jpeg'], 'labels': 'models/densenet_onnx/densenet_labels.txt', 'model': 'models/densenet_onnx/1/model.onnx', 'model_metadata': 'models/densenet_onnx/config.pbtxt', 'model_repository': 's3://https://s3.edge-farm.agrifood-tef.edvsz.hs-osnabrueck.de:443/a0713a6a-17fd-4de5-8e63-3264e9ea5b16/models'}
2024-07-04 14:19:28.767 INFO client - get_benchmark_job_results: Results for benchmark job 'a0713a6a-17fd-4de5-8e63-3264e9ea5b16' are not yet available.
2024-07-04 14:19:29.831 INFO client - get_benchmark_job_results: Results for benchmark job 'a0713a6a-17fd-4de5-8e63-3264e9ea5b16' are not yet available.


### 6. Ergebnisse abrufen und verarbeiten

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F

from collections import defaultdict

In [ ]:
# Benchmark results
benchmark_results = pd.DataFrame(benchmark_results)
print(benchmark_results)

['2024-07-04T16:19:27.463079', '2024-07-04T16:19:27.463375', '2024-07-04T16:19:27.660321', '2024-07-04T16:19:28.665821', '2024-07-04T16:19:29.654672'] [0.0, 0.0, 0.0, 0.0, 0.0] [0.026200936653704862, 0.026200936653704862, 0.02916685742779028, 0.02917581490663819, 0.029554330592676564] [44.75, 44.75, 44.312, 44.25, 44.468]


In [86]:
inference_results_table = np.stack(inference_results)

final_inference_results = defaultdict(list)
for sample_predictions in inference_results_table:
    logits = sample_predictions[:, 0].astype(float)
    probabilities = F.softmax(torch.tensor(logits), dim=0)

    predicted_classes = sample_predictions[:, -1]
    predicted_class_index = probabilities.argmax()
    predicted_probability = probabilities.max()
    predicted_class = predicted_classes[predicted_class_index]

    final_inference_results["class"].append(predicted_class)
    final_inference_results["probability"].append(
        predicted_probability.item() * 100
    )

inference_results_df = pd.DataFrame(final_inference_results)
print(inference_results_df)

ANALOG CLOCK
ANALOG CLOCK
ANALOG CLOCK
